In [ ]:
#Se importan paquterías
import os
import numpy as np 
from netCDF4 import Dataset

##Se obtienen paths
#netcdf
user_dir=os.path.expanduser('~')
path_nc_mod1=os.path.join(user_dir,'IOA','wrf_val','Temp_corr','netcdfs','wrfout_d01_2020-02-16_00.nc')

#Se lee netcdf para obtener lat y lon
with Dataset(path_nc_mod1,'r') as f_n1:
    var=f_n1.variables
    lon=var.get('XLONG')[0,:,:]
    lat=var.get('XLAT')[0,:,:]
    print(lat.shape)

##Calculo del albedo

#dia del año desde el 1 de enero del 2020
dia=47
#se ajusta la fase con base en el dia del año 
#correspondiente al equinoccio de primavera
dia_fase=dia-79
#numero de dias en el año 2020
dias_año=366
dia_en_grados=360*(dia_fase/dias_año)
# el angulo de inclinacion terrestre
eps=23.5
#ecuacion del tiempo (correccion del tiempo medio respecto al tiempo solar)
eot=(9.87*np.sin(np.deg2rad(2*dia_en_grados)))-\
(7.53*np.cos(np.deg2rad(dia_en_grados)))-\
(1.5*np.sin(np.deg2rad(dia_en_grados)))
#tiempo local en grados= 15*(hora local-hora greenwich)
lstm=15*(-6) #aqui se puso en zona horaria cdmx para hacer el mapa
#factor de correccion del tiempo,
#sirve para corregir el tiempo conforme la longitud y dia del año
tc=4*(lon-lstm)+eot 
#se calcula el angulo de declinacion delta
sdelta=np.sin(np.deg2rad(eps))*np.sin(np.deg2rad(dia_en_grados))
delta=np.arcsin(sdelta)

#Se calcula en angulo horario y cenital para cada hora del dia
#Se calcula el albedo horario
Albedo=[]
for hora in range(24):
    #tiempo local solar
    lst=hora+(tc/60)
    #hora angular
    h=15*(lst-12)
    # calculo de ángulo cenital
    mu=((np.sin(np.deg2rad(lat))*np.sin(np.deg2rad(delta))) +\
    (np.cos(np.deg2rad(lat))*np.cos(np.deg2rad(delta))*np.cos(np.deg2rad(h))))
    # Paramterizaicón que calcula el albedo horario
    A=0.037/(1.1*abs(mu**1.4)+0.15)
    Albedo.append(A)
Albedo=np.array(Albedo)

#Se guarda en un NetCDF la matriz de albedo del día en cuestión
new_nc=os.path.join(user_dir,'IOA','hycom','Albedo_2020-02-16_00.nc')
with Dataset(new_nc,'w',format='NETCDF4') as fm:
    fm.createDimension('south_north',262)
    fm.createDimension('west_east',338)
    fm.createDimension('Time',24)
    # se copia atributos y variable de la
    fm.createVariable('XLAT', 'f4', ('south_north','west_east'))
    fm.createVariable('XLONG', 'f4', ('south_north','west_east'))
    fm.createVariable('Albedo', 'f4', ('Time','south_north','west_east'))
    fm.variables['XLAT'][:] = lat
    fm.variables['XLONG'][:] = lon
    fm.variables['Albedo'][:] = Albedo
     

#Referencias
#Script de calculo de albedo calcula_albedo_v8.mat
#ManualHycom2017_V1-0
#Taylor et al. (1996)
#https://www.pveducation.org/pvcdrom/properties-of-sunlight/solar-time


(262, 338)
